# <div align="center"><b> ETIQUETADO - PROYECTO FINAL </b></div>

<div align="right">📝 <em><small><font color='Gray'>Nota:</font></small></em></div>

<div align="right"> <em><small><font color='Gray'> La funcionalidad de visualización de jupyter notebooks en <a href="https://github.com/" target="_blank">github</a> es solamente un preview.</font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Para mejor visualización se sugiere utilizar el visualizador recomendado por la comunidad: <a href="https://nbviewer.org/" target="_blank">nbviewer</a></font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Puedes a acceder al siguiente enlace para ver este notebook en dicha página: <a href="https://nbviewer.org/ruta/de/archivo.ipynb">Ruta archivo</a></font></small></em> </div>

* * *

<style>
/* Limitar la altura de las celdas de salida en html */
.jp-OutputArea.jp-Cell-outputArea {
    max-height: 500px;
}
</style>

✋ <em><font color='DodgerBlue'>Importaciones:</font></em> ✋

In [2]:
import os, sys, logging, json, shutil, zipfile, copy, datetime, re, requests
from pathlib import Path
from typing import List, Dict, Tuple, Any, Optional, Literal
from concurrent.futures import ProcessPoolExecutor

sys.path.append(os.path.abspath("../"))  # Agregar el directorio padre al path

from dotenv import load_dotenv
load_dotenv("../.env.dev")

from pprint import pprint


from cvat_sdk import make_client
from cvat_sdk.core.proxies.types import Location

from pymongo import UpdateOne

from pycocotools.coco import COCO

import matplotlib.pyplot as plt

import layoutparser as lp

OPENCV_IO_MAX_IMAGE_PIXELS = 50000 * 50000  # Para imágenes grandes, ej: barrio3Ombues_20180801_dji_pc_3cm.jpg
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = str(OPENCV_IO_MAX_IMAGE_PIXELS)

import cv2 as cv
import numpy as np

from fastkml import kml
import kml2geojson

import geopandas as gpd
from shapely.geometry import Point, Polygon, box
import pandas as pd

from tqdm import tqdm
from apps_config.settings import Config
from apps_com_db.mongodb_client import MongoDB
from apps_com_s3.minio_client import S3Client
from apps_utils.logging import Logging

🔧 <em><font color='tomato'>Configuraciones:</font></em> 🔧


In [3]:
CONFIG = Config().config_data
DB = MongoDB().db
MINIO_CLIENT = S3Client().client
LOGGER = Logging().logger

MINIO_BUCKET = CONFIG["minio"]["bucket"]

MINIO_PATCHES_FOLDER = CONFIG["minio"]["paths"]["patches"]
download_folder = Path(CONFIG["folders"]["download_folder"])
DOWNLOAD_TASK_FOLDER = download_folder / "tasks"
DOWNLOAD_JOB_FOLDER = download_folder / "jobs"
DOWNLOAD_TEMP_FOLDER = download_folder / "temp"
DOWNLOAD_COCO_ANNOTATIONS_FOLDER = download_folder / "coco_annotations"
DOWNLOAD_IMAGES_FOLDER = download_folder / "images"
DOWNLOAD_PATCHES_FOLDER = download_folder / "patches"
DOWNLOAD_CUTOUTS_FOLDER = download_folder / "cutouts"
DOWNLOAD_CUTOUTS_METADATA_FOLDER = download_folder / "cutouts_metadata"
DOWNLOAD_GOOGLE_MAPS_FOLDER = download_folder / "google_maps"
KMLS_FOLDER = download_folder / "kmls"
GEOJSON_FOLDER = download_folder / "geojson"

LOGGER.info("Configuración cargada correctamente.")

2025-05-17 17:22:17,378 - root - INFO - <module> - Configuración cargada correctamente.


<!-- Colab -->
<!-- <div align="center"><img src="https://drive.google.com/uc?export=view&id=1QSNrTsz1hQbmZwpgwx0qpfpNtLW19Orm" width="600" alt="Figura 1: A data scientist is working on word generation using the Lord of the Rings lore. The image is dark and moody, with a focus on the scientist's computer screen. The screen displays a visualization the one ring, with a map of Middle Earth in the background. - Generada con DALL-E3"></div> -->

<!-- <div align="center"><img src="./ceia-materia/resources/portada.jpeg" width="600" alt="Figura 1: A data scientist playing with convolutional neural networks. - Generada con Microsoft Image Creator"></div>

<div align="center"><small><em>Figura 1: A data scientist playing with convolutional neural networks. - Generada con Microsoft Image Creator</em></small></div> -->

<div align="center">✨Datos del proyecto:✨</div>

<p></p>

<div align="center">

| Subtitulo       | Etiquetado                                                                                                                             |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| **Descrpción**  | Herramientas y scripts para el etiquetado y guardado de los datos                                                                      |
| **Integrantes** | Bruno Masoller (brunomaso1@gmail.com)                                                                                                  |

</div>

## Consinga

El objetivo de este proyecto es brindar herramientas y scripts para el etiquetado y guardado de los datos.

## Resolución

Utilidades de Ultralytics:

- [https://docs.ultralytics.com/es/usage/simple-utilities/](https://docs.ultralytics.com/es/usage/simple-utilities/)

### Anotaciones

Para las anotaciones, entre varios formatos estudiados, se eligió el formato de COCO.

- [https://roboflow.com/formats/coco-json](https://roboflow.com/formats/coco-json)
- [https://docs.voxel51.com/recipes/convert_datasets.html](https://docs.voxel51.com/recipes/convert_datasets.html)
- [https://stackoverflow.com/questions/75927857/how-to-convert-coco-json-to-yolov8-segmentation-format](https://stackoverflow.com/questions/75927857/how-to-convert-coco-json-to-yolov8-segmentation-format)

### Interacción con google maps

#### Generación de mapa

Procesamiento de archivos KML:
- `fastkml` $\rightarrow$ [https://fastkml.readthedocs.io/en/latest/quickstart.html](https://fastkml.readthedocs.io/en/latest/quickstart.html)

Conversión de archivos KML a GeoJSON:
- `kml2geojson` $\rightarrow$ [https://pypi.org/project/kml2geojson/](https://pypi.org/project/kml2geojson/) | [https://github.com/mrcagney/kml2geojson](https://github.com/mrcagney/kml2geojson)

Visualización:
- `folium` $\rightarrow$ [https://python-visualization.github.io/folium/latest/](https://python-visualization.github.io/folium/latest/)
- `geojson.io` $\rightarrow$ [https://geojson.io/#map=2/0/20](https://geojson.io/#map=2/0/20)

Trabajo con archivos GeoJSON:
- `geojson` $\rightarrow$ [https://github.com/jazzband/geojson](https://github.com/jazzband/geojson) | [https://geojson.org/](https://geojson.org/)
- `geopandas` $\rightarrow$ [https://geopandas.org/](https://geopandas.org/)
- `shapely` $\rightarrow$ [https://shapely.readthedocs.io/en/latest/manual.html](https://shapely.readthedocs.io/en/latest/manual.html)

In [ ]:
def download_kmz_from_gmaps(
    base_url: str = CONFIG["google_maps"]["base_url"],
    mid: str = CONFIG["google_maps"]["mid"],
    output_filename: Optional[Path] = None,
) -> Optional[Path]:
    """Descarga un archivo KMZ desde Google Maps y lo descomprime.

    Este método utiliza la configuración proporcionada para construir la URL de descarga
    del archivo KMZ desde Google Maps. Una vez descargado, el archivo se descomprime
    y se guarda en la carpeta especificada.

    Args:
        filename (str, optional): Ruta donde se guardará el archivo KMZ descargado.
                                  Si no se proporciona, se utiliza una ruta temporal.

    Returns:
        str: Ruta del archivo KML descargado y descomprimido.

    Raises:
        Exception: Si ocurre un error al acceder a la URL de descarga.
    """
    url = f"{base_url}?mid={mid}"
    if not output_filename:
        Path(DOWNLOAD_TEMP_FOLDER).mkdir(parents=True, exist_ok=True)
        output_filename = DOWNLOAD_TEMP_FOLDER / "google_maps.kmz"
    try:
        response = requests.get(url)
        response.raise_for_status()
        with open(output_filename, "wb") as f:
            f.write(response.content)
        LOGGER.debug(f"Archivo KMZ descargado y guardado en {output_filename}.")

        extract_path = (
            DOWNLOAD_GOOGLE_MAPS_FOLDER / f"google_maps_{mid}_{datetime.datetime.now().strftime("%Y%m%d%H%M%S")}.kmz"
        )

        # Descomprimir el archivo KMZ
        with zipfile.ZipFile(output_filename, "r") as zip_ref:
            zip_ref.extractall(extract_path)
        LOGGER.debug(f"Archivo KMZ descomprimido en {extract_path}.")

        # Clear the temporary file
        os.remove(output_filename)

        return f"{extract_path}/doc.kml"

    except requests.exceptions.HTTPError as err:
        raise Exception(f"Error al acceder a {url}. Razón: {err}")

In [ ]:
def convert_kml_to_geojson(kml_file_path: Path, geojson_file_path: Optional[Path]) -> Optional[Path]:
    """
    Convierte un archivo KML a formato GeoJSON.

    Args:
        kml_file_path (str): Ruta al archivo KML de entrada.
        geojson_file_path (str): Ruta donde se guardará el archivo GeoJSON convertido.
    """
    try:
        geojson = kml2geojson.main.convert(kml_file_path)[0]
        LOGGER.info(f"Conversión exitosa de '{kml_file_path}' a '{geojson_file_path}'.")

        if not geojson_file_path:
            Path(DOWNLOAD_GOOGLE_MAPS_FOLDER).mkdir(parents=True, exist_ok=True)
            geojson_file_path = DOWNLOAD_GOOGLE_MAPS_FOLDER / f"{kml_file_path.stem}.geojson"
        # Guardar el archivo GeoJSON
        with open(geojson_file_path, "w") as f:
            json.dump(geojson, f, indent=4)
            return geojson_file_path
        LOGGER.info(f"Archivo GeoJSON guardado en '{geojson_file_path}'.")

    except FileNotFoundError:
        LOGGER.error(f"Error: Archivo KML no encontrado en '{kml_file_path}'.")
    except ValueError as e:
        LOGGER.error(f"Error de valor: {e}")
    except Exception as e:
        LOGGER.error(f"Ocurrió un error inesperado: {e}")
    return None

In [ ]:
def create_geojson_from_annotation(
    pic_name: str,
    coco_annotation: Dict[str, Any],
    jgw_data: Dict[str, Any],
    output_filename: Optional[Path] = None,
    upload_to_drive: bool = False,
    geo_sistema_referencia: str = CONFIG["georeferenciacion"]["codigo_epsg"],
) -> gpd.GeoDataFrame:
    """
    Crea un GeoDataFrame a partir de las anotaciones COCO y datos de georreferenciación.

    Args:
        pic_name (str): Nombre de la imagen para la cual se generará el GeoJSON.
        coco_annotation (Dict[str, Any]): Anotaciones en formato COCO que incluyen categorías y bounding boxes.
        jgw_data (Dict[str, Any]): Datos de georreferenciación provenientes de un archivo JGW.
        output_filename (Optional[Path], optional): Ruta donde se guardará el archivo GeoJSON. Defaults to None.
        upload_to_drive (bool, optional): Indica si el archivo GeoJSON debe subirse a Google Drive. Defaults to False.
        geo_sistema_referencia (str, optional): Código EPSG del sistema de referencia geográfico. Defaults to CONFIG["georeferenciacion"]["codigo_epsg"].

    Raises:
        NotImplementedError: Si se solicita subir el archivo a Google Drive, pero la funcionalidad no está implementada.

    Returns:
        gpd.GeoDataFrame: GeoDataFrame que contiene las geometrías y propiedades de las anotaciones.

    Ejemplo de uso:

        >>> image_name = "8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm"
        >>> patch_name = "8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_patch_0"
        >>> annotations_field = "cvat"
        >>> pic_name = image_name
        >>> if pic_name == image_name:
        >>>     coco_annotations = load_coco_annotation_from_mongodb(
        ...         field_name=annotations_field, image_name=image_name
        ...     )
        >>>     jgw_data = load_jgw_file_from_mongodb(image_name=image_name)
        >>> else:
        >>>     coco_annotations = load_coco_annotation_from_mongodb(
        ...         field_name=annotations_field, patch_name=patch_name
        ...     )
        >>>     jgw_data = load_jgw_file_from_mongodb(patch_name=patch_name)
        >>> gdf = create_geojson_from_annotation(
        ...     pic_name=pic_name,
        ...     coco_annotation=coco_annotations,
        ...     jgw_data=jgw_data,
        ...     output_filename=DOWNLOAD_TEMP_FOLDER / f"{pic_name}.geojson",
        ... )

        >>> # Cambiar proyectar en otro sistema de coordenadas
        >>> # gdf_crs = gdf.to_crs("EPSG:4326")
        >>> # gdf_crs.to_file(
        ... #     DOWNLOAD_TEMP_FOLDER / f"{pic_name}_4326.geojson",
        ... #     driver="GeoJSON",
        ... # )
    """
    # 1 - Configuraciones generales
    category_map = {cat["id"]: cat["name"] for cat in coco_annotation["categories"]}

    # 2 - Obtener el id de la imagen en las anotaciones
    image_id = get_image_id_from_annotations(pic_name, coco_annotation)

    # 3 - Obtener las anotaciones de la imagen
    annotations = [ann for ann in coco_annotation["annotations"] if ann["image_id"] == image_id]
    if not annotations:
        LOGGER.warning(f"No se encontraron anotaciones para la imagen {pic_name}.")
        return gpd.GeoDataFrame()

    # 4 - Preparar listas para almacenar los datos
    geometries = []
    properties = []

    # 5 - Para cada anotación, obtener el bbox y la categoría
    for annotation in annotations:
        bbox = annotation["bbox"]
        category_name = category_map.get(annotation["category_id"], "Sin categoría")

        # 5.1 - Convertir el bbox a coordenadas geográficas utilizando los datos del archivo JGW
        global_coordinates = convert_bbox_image_to_world(bbox, jgw_data)

        # 5.2 - Obtener el centroide del bbox
        x_coords = [
            global_coordinates["tl"][0],
            global_coordinates["tr"][0],
            global_coordinates["br"][0],
            global_coordinates["bl"][0],
        ]
        y_coords = [
            global_coordinates["tl"][1],
            global_coordinates["tr"][1],
            global_coordinates["br"][1],
            global_coordinates["bl"][1],
        ]
        centroid_x = sum(x_coords) / len(x_coords)
        centroid_y = sum(y_coords) / len(y_coords)
        centroid = (centroid_x, centroid_y)

        # 5.3 - Crear un objeto Point de Shapely con las coordenadas del centroide
        point = Point(centroid)

        # 5.4 - Guardar la geometría y propiedades
        geometries.append(point)
        properties.append(
            {
                "category": category_name,
                "annotation_id": annotation.get("id", None),
                "bbox_x": bbox[0],
                "bbox_y": bbox[1],
                "bbox_width": bbox[2],
                "bbox_height": bbox[3],
                "global_tl_x": global_coordinates["tl"][0],
                "global_tl_y": global_coordinates["tl"][1],
                "global_br_x": global_coordinates["br"][0],
                "global_br_y": global_coordinates["br"][1],
            }
        )

    # 6 - Crear un DataFrame con las propiedades
    properties_df = pd.DataFrame(properties)

    # 7 - Crear un GeoDataFrame con las geometrías y propiedades
    gdf = gpd.GeoDataFrame(properties_df, geometry=geometries)

    # 8 - Configurar el sistema de coordenadas (CRS)
    gdf.crs = geo_sistema_referencia

    # 9 - Guardar como GeoJSON si se proporciona un nombre de archivo
    if output_filename:
        output_path = output_filename
        Path(output_path.parent).mkdir(parents=True, exist_ok=True)
        gdf.to_file(output_path, driver="GeoJSON")
        LOGGER.info(f"GeoJSON guardado en {output_path}")

        # 10 - Opcional: Subir a Google Drive si se solicita
        if upload_to_drive:
            # Aquí iría tu código para subir a Drive
            raise NotImplementedError("Subida a Google Drive no implementada.")
    return gdf

In [ ]:
# Se está generando el archivo con unos tags incorrectos.
def create_kml_from_geojson(
    gdf: gpd.GeoDataFrame,
    kml_filename: Optional[Path] = None,
    category_column: str = "category",
    target_category: Optional[str] = "palmera",
    reproject: bool = True,
):
    """
    Crea un archivo KML a partir de un GeoDataFrame.

    Este método toma un GeoDataFrame con geometrías y propiedades, filtra las categorías
    deseadas, y genera un archivo KML con las geometrías y propiedades correspondientes.

    Args:
        gdf (gpd.GeoDataFrame): GeoDataFrame que contiene las geometrías y propiedades.
        kml_filename (Optional[Path], optional): Ruta donde se guardará el archivo KML.
                                                 Si no se proporciona, se utiliza una ruta predeterminada.
                                                 Defaults to None.
        category_column (str, optional): Nombre de la columna que contiene las categorías.
                                         Defaults to "category".
        target_category (Optional[str], optional): Categoría objetivo que se desea incluir en el KML.
                                                   Si no se proporciona, se incluyen todas las categorías.
                                                   Defaults to "palmera".
        reproject (bool, optional): Si se debe reproyectar el GeoDataFrame a EPSG:4326 (WGS84).
                                    Defaults to True.

    Raises:
        ValueError: Si ocurre un error al reproyectar el GeoDataFrame.

    Returns:
        None: El archivo KML se guarda en la ubicación especificada o predeterminada.

    Ejemplo de uso:

        >>> image_name = "8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm"
        >>> patch_name = "8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_patch_0"
        >>> annotations_field = "cvat"
        >>> pic_name = image_name
        >>> if pic_name == image_name:
        >>>     coco_annotations = load_coco_annotation_from_mongodb(
        ...         field_name=annotations_field, image_name=image_name
        ...     )
        >>>     jgw_data = load_jgw_file_from_mongodb(image_name=image_name)
        >>> else:
        >>>     coco_annotations = load_coco_annotation_from_mongodb(
        ...         field_name=annotations_field, patch_name=patch_name
        ...     )
        >>>     jgw_data = load_jgw_file_from_mongodb(patch_name=patch_name)
        >>> gdf = create_geojson_from_annotation(
        ...     pic_name=pic_name,
        ...     coco_annotation=coco_annotations,
        ...     jgw_data=jgw_data,
        ...     output_filename=DOWNLOAD_TEMP_FOLDER / f"{pic_name}.geojson",
        ... )
        >>> create_kml_from_geojson(
        ...     gdf=gdf,
        ...     kml_filename=KMLS_FOLDER / f"{pic_name}.kml",
        ...     category_column="category",
        ...     target_category=None
        ... )
    """
    k = kml.KML()
    ns = "{http://www.opengis.net/kml/2.2}"

    # Crear un documento KML
    palm_document = kml.Document(ns, id="docid", description="PalmTrees")
    k.append(palm_document)

    # Crear una carpeta para las palmeras
    palm_folder = kml.Folder(ns, id="palmeras_folder", name="Palmeras")
    palm_document.append(palm_folder)

    if target_category:
        palm_gdf = gdf[gdf[category_column] == target_category].copy()
    else:
        palm_gdf = gdf.copy()

    if palm_gdf.empty:
        LOGGER.warning(
            f"No se encontraron elementos con la categoría '{target_category}'. No se creará el archivo KML."
        )
        return

    # Reproyectar a WGS84 (EPSG:4326) si no está ya en ese CRS
    if reproject and palm_gdf.crs is not None and palm_gdf.crs != "EPSG:4326":
        try:
            palm_gdf = palm_gdf.to_crs(epsg=4326)
            LOGGER.debug("GeoDataFrame reproyectado a EPSG:4326 para el KML.")
        except Exception as e:
            raise ValueError(
                f"Error al reproyectar el GeoDataFrame a EPSG:4326: {e}. Asegúrate de que el CRS original sea válido."
            )

    for index, row in palm_gdf.iterrows():
        if row.geometry.geom_type == "Point":
            coords = (row.geometry.x, row.geometry.y)
            point = Point(coords)
            p = kml.Placemark(ns, id=f"palmera_{index}", name=f"{row.category}", geometry=point)
            palm_folder.append(p)
        else:
            LOGGER.warning(f"La geometría del elemento con índice {index} no es un Point. No se agregará al KML.")

    if kml_filename:
        Path(kml_filename).parent.mkdir(parents=True, exist_ok=True)
    else:
        Path(KMLS_FOLDER).mkdir(parents=True, exist_ok=True)
        kml_filename = KMLS_FOLDER / f"kml_{target_category}_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.kml"
    try:
        k.write(kml_filename)
        LOGGER.info(f"Archivo KML guardado en {kml_filename}")
    except Exception as e:
        LOGGER.error(f"Error al guardar el archivo KML: {e}")

In [42]:
def load_gdf_from_file(
    file_path: Path,
    crs: Optional[str] = None,
) -> gpd.GeoDataFrame:
    """Carga un archivo GeoJSON y lo convierte a un GeoDataFrame.

    Args:
        file_path (str): Ruta al archivo GeoJSON.
        driver (str, optional): Controla el formato del archivo. Defaults to "GeoJSON".
        crs (str, optional): Sistema de referencia de coordenadas. Defaults to None.

    Returns:
        gpd.GeoDataFrame: GeoDataFrame que contiene los datos del archivo.
    """
    gdf = gpd.read_file(file_path)
    if crs:
        gdf.crs = crs
    return gdf

#### Procesamiento de mapas

In [ ]:
def process_single_patch(
    imagen: Dict[str, Any],
    patch: Dict[str, Any],
    gdf: gpd.GeoDataFrame,
    bbox_size: Tuple[float, float] = (CONFIG["bbox_size"]["width"], CONFIG["bbox_size"]["height"]),
) -> Tuple[Dict[str, Any], List[Dict[str, Any]]]:
    image = {
        "width": patch["width"],
        "height": patch["height"],
        "file_name": f"{patch["patch_name"]}.jpg",
        "date_captured": imagen["date_captured"].strftime("%Y-%m-%d %H:%M:%S"),
    }
    annotations = []
    jgw_data = imagen.get("jgw_data")
    if not jgw_data:
        LOGGER.warning(f"No se encontró el archivo JGW para la imagen {imagen['name']}.")
        return image, annotations

    # Obtener las coordenadas del parche
    x_start, y_start, patch_width, patch_height = (
        patch["x_start"],
        patch["y_start"],
        patch["width"],
        patch["height"],
    )
    # Esquinas del parche dentro de la imagen
    esquinas_imagen = [
        (x_start, y_start),  # esquina superior izquierda
        (x_start + patch_width, y_start),  # esquina superior derecha
        (x_start + patch_width, y_start + patch_height),  # esquina inferior derecha
        (x_start, y_start + patch_height),  # esquina inferior izquierda
    ]

    # Convertir las coordenadas del parche a coordenadas globales
    esquinas_mundo = [convert_point_image_to_world(punto, jgw_data=jgw_data) for punto in esquinas_imagen]

    poligono_parche = Polygon(esquinas_mundo)

    # Filtrar los puntos del GeoDataFrame que están dentro del polígono del parche
    puntos_en_parche = gdf[gdf.geometry.within(poligono_parche)]

    if puntos_en_parche.empty:
        LOGGER.debug(f"No se encontraron puntos dentro del parche {patch['patch_name']}.")
        return image, annotations

    # Procesar cada punto encontrado del parche
    puntos_en_parche.reset_index(inplace=True, drop=True)
    for index, row in puntos_en_parche.iterrows():
        punto_mundo = (row.geometry.x, row.geometry.y)

        # Convertir a coordenadas de imagen
        punto_imagen = convert_point_world_to_image(punto_mundo, jgw_data)

        # Convertir a coordenadas locales del parche
        punto_parche = convert_point_image_to_patch(punto_imagen, x_start, y_start, patch_width, patch_height)

        # Crear el bounding box
        bbox_ancho, bbox_alto = bbox_size
        x_centro, y_centro = punto_parche

        # Asegurarse que el bbox no exceda los límites del parche
        x_min = max(0, x_centro - bbox_ancho / 2)
        y_min = max(0, y_centro - bbox_alto / 2)
        x_max = min(patch_width, x_centro + bbox_ancho / 2)
        y_max = min(patch_height, y_centro + bbox_alto / 2)

        # Calcular dimensiones finales del bbox
        ancho = x_max - x_min
        alto = y_max - y_min
        area = ancho * alto

        category_name = "palmera-google-maps"
        annotation = {
            "id": index + 1,
            "segmentation": [],
            "iscrowd": 0,
            "attributes": {
                "occluded": False,
                "rotation": 0.0,
            },
            "category_name": category_name,
            "area": area,
            "bbox": [x_min, y_min, ancho, alto],
        }

        annotations.append(annotation)

    return image, annotations

In [ ]:
def create_annotations_from_geojson(
    gdf: gpd.GeoDataFrame,
    output_filename: Optional[Path] = None,
    use_parallel: bool = True,
    max_workers: int = 10,
) -> Dict[str, Any]:
    coco_annotations = {
        "info": CONFIG["coco_dataset"]["info"],
        "licenses": CONFIG["coco_dataset"]["licenses"],
        "categories": CONFIG["google_maps"]["categories"],
        "images": [],
        "annotations": [],
    }

    category_map = {cat["name"]: cat["id"] for cat in coco_annotations["categories"]}

    coco_images = []
    image_annotations = []
    imagenes = DB.get_collection("imagenes")

    # Consulta con agregación para filtrar imágenes y sus patches
    pipeline = [
        # Filtrar imágenes donde downloaded = true
        {"$match": {"downloaded": True}},
        # Crear un nuevo campo 'patches_filtrados' que contenga solo los patches donde is_white = false
        {
            "$addFields": {
                "patches_filtrados": {
                    "$filter": {"input": "$patches", "as": "patch", "cond": {"$eq": ["$$patch.is_white", False]}}
                }
            }
        },
        # Filtrar para incluir solo imágenes que tienen al menos un patch válido
        {"$match": {"patches_filtrados.0": {"$exists": True}}},
        # Opcionalmente: proyectar solo campos necesarios con $project (mejorar optimización, pero queda hardcodeado)
    ]

    filtered_images = list(imagenes.aggregate(pipeline))
    LOGGER.debug(f"Se encontraron {len(filtered_images)} imágenes con parches no blancos.")

    # Creamos tareas asíncronas para cada imagen y parche
    tareas = [(imagen, patch) for imagen in filtered_images for patch in imagen["patches_filtrados"]]
    LOGGER.debug(f"Se encontraron {len(tareas)} tareas para procesar.")

    if tareas:
        annotation_images = []
        if use_parallel:
            with ProcessPoolExecutor(max_workers=max_workers) as executor:
                futures = [(executor.submit(process_single_patch, imagen, patch, gdf)) for imagen, patch in tareas]
                for future in tqdm(futures, desc="Procesando parches"):
                    try:
                        image, annotations = future.result()
                        if image and annotations:
                            annotation_images.append((image, annotations))
                    except Exception as e:
                        LOGGER.error(f"Error procesando el parche: {e}")
        else:
            for imagen, patch in tqdm(tareas, desc="Procesando parches"):
                try:
                    image, annotations = process_single_patch(imagen, patch, gdf)
                    if image and annotations:
                        annotation_images.append((image, annotations))
                except Exception as e:
                    LOGGER.error(f"Error procesando el parche: {e}")

        for id, image, annotations in enumerate(annotation_images):
            image_id = id + 1
            image = {"id": image_id, **image}

            annotations = [
                {
                    **annotation,
                    "image_id": image_id,
                    "category_id": category_map[annotation["category_name"]],
                }
                for annotation in annotations
            ]

            coco_images.append(image)
            image_annotations.append(annotations)

        coco_annotations["images"] = coco_images
        coco_annotations["annotations"] = image_annotations

        if output_filename:
            with open(output_filename, "w") as f:
                json.dump(coco_annotations, f, indent=4)
                LOGGER.debug(f"Anotaciones guardadas en {output_filename}")

        pprint(coco_annotations)
    else:
        LOGGER.warning("No se encontraron tareas para procesar.")

In [45]:
# gdf = load_gdf_from_file(DOWNLOAD_TEMP_FOLDER / "google_maps.geojson", crs=CONFIG["georeferenciacion"]["codigo_epsg"])
# output_filename = None
# use_parallel = False
# max_workers = 10

In [46]:
def merge_annotations():
    pass

### Ejemplo de carga de anotaciones

- Desde CVAT:
```python
coco_annotations = load_annotations_from_cvat(task_id=8)
save_coco_annotations(coco_annotations=coco_annotations, field_name="cvat")
```

- Desde un archivo:
```python
coco_annotations = load_annotations_from_file(file_path="cvat.json")
save_coco_annotations(coco_annotations=coco_annotations, field_name="cvat")
```

In [47]:
# patches_list = [
#     "8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_patch_0",
#     "8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_patch_2",
# ]
# images_list = ["8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm", "AntelArena_20200804_dji_pc_5c"]
# coco_annotations = load_coco_annotations_from_mongodb(field_name="cvat", images_names=images_list, clean_files=False)
# save_coco_annotations(coco_annotations, "test")

### Fixes

In [ ]:
def fix_image_width_height(image_id: str, width: int, height: int) -> bool:
    """Actualiza el ancho y alto de una imagen en la base de datos.

    Args:
        image_id (str): Identificador de la imagen a actualizar.
        width (int): Nuevo ancho de la imagen.
        height (int): Nuevo alto de la imagen.

    Returns:
        bool: True si la actualización fue exitosa, False en caso contrario.
    """
    try:
        imagenes = DB.get_collection("imagenes")
        result = imagenes.update_one(
            {"id": image_id},
            {
                "$set": {
                    "width": width,
                    "height": height,
                }
            },
        )
        if result.modified_count > 0:
            LOGGER.debug(f"Ancho y alto de la imagen {image_id} actualizados correctamente.")
            return True
        else:
            LOGGER.warning(f"No se encontraron cambios para la imagen {image_id}.")
            return False
    except Exception as e:
        LOGGER.error(f"Error al actualizar el ancho y alto de la imagen {image_id}: {e}")
        return False

In [ ]:
def fix_width_height_images():
    """Corrige el ancho y alto de las imágenes en la base de datos.

    Esta función descarga las imágenes desde MinIO, obtiene sus dimensiones (ancho y alto),
    actualiza estos valores en la base de datos y elimina las imágenes descargadas localmente.

    Pasos realizados:
    1. Configura el logger para registrar las actualizaciones.
    2. Obtiene todas las imágenes almacenadas en la base de datos.
    3. Descarga cada imagen desde MinIO si está marcada como descargada.
    4. Calcula las dimensiones de la imagen descargada.
    5. Actualiza las dimensiones en la base de datos.
    6. Elimina la imagen descargada localmente.

    Raises:
        OSError: Si ocurre un error al eliminar la imagen descargada localmente.
    """

    set_log_to_file(f"updates_{datetime.date.today()}.log")
    LOGGER.setLevel(logging.INFO)
    LOGGER.info("Iniciando actualización de imágenes y parches.")
    # Actualizar el width y height de las imágenes en la base de datos
    # 1 - Obtener todas las imágenes de la base de datos
    imagenes = DB.get_collection("imagenes")

    # 2 - Para cada imagen:
    for image in imagenes.find():
        image_id = image["id"]
        downloaded: bool = image["downloaded"]
        if not downloaded:
            LOGGER.warning(f"La imagen {image_id} no está descargada, se omitirá.")
            continue
        LOGGER.info(f"Actualizando imagen {image_id}...")
        # 2.1 - Descargar la imagen desde MinIO

        jpg_path = download_image_from_minio(image_id)
        LOGGER.info(f"Imagen {image_id} descargada correctamente.")
        # 2.2 - Obtener el ancho y alto de la imagen descargada
        img = cv.imread(jpg_path)
        height, width = img.shape[:2]
        # 2.3 - Actualizar el ancho y alto de la imagen en la base de datos
        fix_image_width_height(image_id, width, height)
        LOGGER.info(f"Ancho y alto de la imagen {image_id} actualizados correctamente.")
        # 2.4 - Eliminar la imagen descargada
        try:
            os.remove(jpg_path)
            LOGGER.info(f"Imagen {jpg_path} eliminada correctamente.")
        except OSError as e:
            LOGGER.warning(f"Error al eliminar la imagen {jpg_path}: {e}")
            continue
    LOGGER.info("Actualización de imágenes y parches finalizada.")

In [ ]:
def fix_dates_images():
    set_log_to_file(f"fix_dates_images_{datetime.date.today()}.log")
    LOGGER.setLevel(logging.INFO)
    LOGGER.info("Iniciando actualización de fechas de imágenes.")
    # Actualizar el width y height de las imágenes en la base de datos
    # 1 - Obtener todas las imágenes de la base de datos
    imagenes = DB.get_collection("imagenes")
    date_format = "%d/%m/%Y"

    # 2 - Para cada imagen:
    for image in imagenes.find():
        # 2.1 - Obtener el título de la imagen
        image_title = image["title"]

        # 2.2 - Extraer la fecha del título de la imagen
        match = re.search(r"\d{2}/\d{2}/\d{4}", image_title)
        date_str = match.group(0) if match else None
        if not date_str:
            LOGGER.warning(f"No se encontró una fecha válida en el título de la imagen {image_title}.")
            continue

        # 2.3 - Actualizar la fecha de captura en la base de datos
        try:
            date_captured = datetime.datetime.strptime(date_str, date_format)
            imagenes.update_one(
                {"id": image["id"]},
                {
                    "$set": {
                        "date_captured": date_captured,
                    }
                },
            )
            LOGGER.info(f"Fecha de captura de la imagen {image_title} actualizada correctamente.")
        except Exception as e:
            LOGGER.warning(f"Error al actualizar la fecha de captura: {e}")

    LOGGER.info("Actualización de fechas de imágenes finalizada.")

In [ ]:
def fix_patch_name():
    """Corrige el nombre de los parches en la base de datos.
    Le saca el .jpg al final del nombre del parche y lo actualiza en la base de datos.
    """
    set_log_to_file(f"fix_patch_name_{datetime.date.today()}.log")
    LOGGER.setLevel(logging.INFO)
    LOGGER.info("Iniciando actualización de nombres de parches.")
    # 1 - Obtener todas las imágenes de la base de datos
    imagenes = DB.get_collection("imagenes")

    # 2 - Para cada imagen:
    for image in imagenes.find():
        image_id = image["id"]
        # 2.1 - Obtener los parches de la imagen
        try:
            patches = image["patches"]
        except KeyError:
            LOGGER.warning(f"No se encontraron parches para la imagen {image_id}.")
            continue
        # 2.2 - Para cada parche:
        for patch in patches:
            patch_name = patch["patch_name"]
            if patch_name.endswith(".jpg"):
                new_patch_name = patch_name[:-4]
                # 2.3 - Actualizar el nombre del parche en la base de datos
                imagenes.update_one(
                    {"id": image_id, "patches.patch_name": patch_name},
                    {"$set": {"patches.$.patch_name": new_patch_name}},
                )
                LOGGER.info(f"Nombre del parche {patch_name} actualizado a {new_patch_name}.")
            else:
                LOGGER.warning(f"El parche {patch_name} no tiene .jpg al final.")
    LOGGER.info("Actualización de nombres de parches finalizada.")